In [1]:
import json
import numpy as np
import pandas as pd

from statistics import mean

In [ ]:
models_name = [
    "bert-base-cased",
    "roberta-base",
    "distilbert-base-cased",
    "microsoft/deberta-base",
    "facebook/FairBERTa",
]

dimensions = [
    "merged"
]

tasks = [
    # "buzzfeed",
    # "politifact",
    # "twittercovidq2",
    # "clef22",
    
    "propaganda",
    "webis",
    "pheme",
    "basil",
    "shadesoftruth",
    "fingerprints",
    "clickbait",
]

is_vanilla_model = [
    True,
    False
]

In [3]:
def preprocess_data_for_evaluation(labels, y_true, y_pred):
    preprocessed_data = {}

    for label in labels.values():
        for idx, i in enumerate(y_true):
            if i == label:
                preprocessed_data.setdefault(
                    label, 
                    []
                ).append(y_pred[idx])

    return preprocessed_data

In [4]:
def calculate_TPRD(
    unperturbed_labels_distribution,
    perturbed_labels_distribution,
    label
    ):
    unperturbed_label_distribution = unperturbed_labels_distribution[label]
    perturbed_label_distribution = perturbed_labels_distribution[label]

    TP_unperturbed_label_distribution = []
    for prediction in unperturbed_label_distribution:
        if prediction == label:
            TP_unperturbed_label_distribution.append(prediction)

    TP_perturbed_label_distribution = []
    for prediction in perturbed_label_distribution:
        if prediction == label:
            TP_perturbed_label_distribution.append(prediction)

    unperturbed_TPR = len(TP_unperturbed_label_distribution)/(len(unperturbed_label_distribution))
    perturbed_TPR = len(TP_perturbed_label_distribution)/(len(perturbed_label_distribution))

    return abs(unperturbed_TPR - perturbed_TPR)

In [ ]:
def load_predictions(model_name, dimension, task, vanilla_model):
    try:
        if vanilla_model:
            unperturbed_test_preds = np.load(f'../../data/interim/predictions/{model_name}/vanilla/{dimension}/{task}/unperturbed.npy')
            perturbed_test_preds = np.load(f'../../data/interim/predictions/{model_name}/vanilla/{dimension}/{task}/perturbed.npy')
        else:
            unperturbed_test_preds = np.load(f'../../data/interim/predictions/{model_name}/{dimension}/{task}/unperturbed.npy')
            perturbed_test_preds = np.load(f'../../data/interim/predictions/{model_name}/{dimension}/{task}/perturbed.npy')
    except:
        print(f"NO DATA FOR: {model_name} - {dimension} - {task}")
        # continue
    return unperturbed_test_preds, perturbed_test_preds

In [ ]:
for model_name in models_name:
    for task in tasks:
        for dimension in dimensions:
            for is_vanilla in is_vanilla_model:
                labels_file = open(f'../../data/interim/labels.json', "r")
                label2id = json.load(labels_file)[task]
                unperturbed_test_preds, perturbed_test_preds = load_predictions(model_name, dimension, task, is_vanilla)
                
                # Load the test df and drop the data samples without entities for the dimension at hand 
                test_df = pd.read_csv(f'../../data/processed/{dimension}/{task}_test.csv').dropna()
                test_gold = np.array([label2id[str(label)] for label in test_df.labels.tolist()])

                unperturbed_label_distribution = preprocess_data_for_evaluation(
                    label2id, 
                    list(test_gold),
                    list(unperturbed_test_preds)
                    )
                perturbed_label_distribution = preprocess_data_for_evaluation(
                    label2id,
                    list(test_gold),
                    list(perturbed_test_preds)
                    )

                overall_TPRD = []
                for label in label2id.values():
                    overall_TPRD.append(
                        calculate_TPRD(
                            unperturbed_label_distribution,
                            perturbed_label_distribution,
                            label
                        )
                    )
                mean_overall_TPDR = round(mean(overall_TPRD), 3)

                print(f"{model_name} - {task} - {dimension} - is_vanilla={is_vanilla} -> overall_TPRD: {mean_overall_TPDR}")
            print("\n")

bert-base-cased - propaganda - merged - is_vanilla=True -> overall_TPRD: 0.034
bert-base-cased - propaganda - merged - is_vanilla=False -> overall_TPRD: 0.019


bert-base-cased - webis - merged - is_vanilla=True -> overall_TPRD: 0.003
bert-base-cased - webis - merged - is_vanilla=False -> overall_TPRD: 0.037


bert-base-cased - pheme - merged - is_vanilla=True -> overall_TPRD: 0.105
bert-base-cased - pheme - merged - is_vanilla=False -> overall_TPRD: 0.013


bert-base-cased - basil - merged - is_vanilla=True -> overall_TPRD: 0.017
bert-base-cased - basil - merged - is_vanilla=False -> overall_TPRD: 0.042


bert-base-cased - shadesoftruth - merged - is_vanilla=True -> overall_TPRD: 0.025
bert-base-cased - shadesoftruth - merged - is_vanilla=False -> overall_TPRD: 0.01


bert-base-cased - fingerprints - merged - is_vanilla=True -> overall_TPRD: 0.025
bert-base-cased - fingerprints - merged - is_vanilla=False -> overall_TPRD: 0.02


bert-base-cased - clickbait - merged - is_vanilla=True -